In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM
from bs4 import BeautifulSoup
from io import StringIO
import csv
from tqdm import tqdm
import pandas as pd
import numpy as np

/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<00:00, 3.65MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 3.08MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:01<00:00, 1.53MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 156kB/s]
config.json: 100%|██████████| 720/720 [00:00<00:00, 3.11MB/s]
model.safetensors.index.json: 100%|██████████| 92.7k/92.7k [00:00<00:00, 459kB/s]
model-00001-of-00019.safetensors: 100%|██████████| 4.89G/4.89G [24:14<00:00, 3.36MB/s]
model-00002-of-00019.safetensors: 100%|██████████| 4.98G/4.98G [29:02<00:00, 2.86MB/s]
model-00003-of-00019.safetensors: 100%|██████████| 4.98G/4.98G [29:59<00:00, 2.77MB/s]
model-00004-of-00019.safetensors: 100%|██████████| 4.90G/4.90G [17:02<00:00, 4.79MB/s]
model-00005-of-00019.safetensors: 100%|██████████| 4.98G/4.98G [15:01<00:00, 5.53MB/s]
Trying to resume download...










































































































In [17]:




messages = [
    {"role": "user", "content": """<snippet> " 3"," risk-free rate of return ( 2 )"," 1.5% ( 1.5 % )"," 1.5% ( 1.5 % )"," 1.3% ( 1.3 % )" </snippet> 
     On the scale of 1 to 5, how relevant is the above snippet to the following question: 
     <question> What is risk-free rate of return ( 2 ) for 2015 ? </question>
     Consider the following examples and their relevance scores:
     Example 1:
     <snippet> " 6"," fair value per option granted"," $ 15.01"," $ 12.04"," $ 11.03" </snippet> 
     On the scale of 1 to 5, how relevant is the above snippet to the following question: 
     <question> What is credit funds2 for unfunded commitments ? </question>
     Relevance score: 1
     Example 2: 
     <snippet> " 3"," 2011"," 3139"," 91"," 3048" </snippet> 
     On the scale of 1 to 5, how relevant is the above snippet to the following question: 
     <question> What is 2010 for benefit payments ? </question>
     Relevance score: 2
     Example 3: 
     <snippet> " 2"," private equity funds2"," $ 7680"," $ 2778"," $ 8074"," $ 3514" </snippet> 
     On the scale of 1 to 5, how relevant is the above snippet to the following question: 
     <question> What is credit funds2 for unfunded commitments ? </question>
     Relevance score: 3
     Example 4: 
     <snippet> " 3"," credit funds2"," 3927"," 2843"," 3596"," 3568" </snippet> 
     On the scale of 1 to 5, how relevant is the above snippet to the following question: 
     <question> What is credit funds2 for unfunded commitments ? </question>
     Relevance score: 4
     
     Focus on the snippet's relevance to the main topic and details mentioned in the question.
     
     Carefully consider the relevance of the snippet to the question, and provide a score from 1 to 4, where 1 is least relevant and 4 is the most relevant. Use the full range of scores to indicate the degree of relevance.
     
     Respond only with number.
     """},
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

outputs = embed_model.generate(inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<00:00, 2.55MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 3.10MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:01<00:00, 1.74MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 145kB/s]
config.json: 100%|██████████| 720/720 [00:00<00:00, 1.34MB/s]


KeyError: 'mixtral'

In [ ]:
prompt = """
"""

In [2]:
data = pd.read_csv('../data/pseudo_json_dataset_V3_2048.csv')

In [13]:
index = 5
print(data.loc[index, 'question'])


What is 2010 for benefit payments ?


In [14]:
soup = BeautifulSoup(data.loc[index, 'html_content'], 'html.parser')

# Find the table
table = soup.find('table')

In [47]:
output_df = pd.DataFrame(columns=['index', 'question', 'csv_row', 'cosine_similarity', 'dot_product_similarity'])

In [48]:
# with open(filename, 'w') as file:
count = 0
for i, row in tqdm(data[:2000].iterrows()):
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    trows = table.find_all('tr')
    header = trows[0]
    question_embed = embed_model.encode(row['question'])

    for trow in trows[1:]:
        writer = csv.writer(csv_buffer)
        writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
        csv_data = csv_buffer.getvalue().replace('\n', '').replace('  ', '')
        csv_data_embed = embed_model.encode(csv_data)
        csv_buffer = StringIO()
        sim = np.dot(csv_data_embed, question_embed)
        cosine_sim = cosine_similarity(csv_data_embed.reshape(1, -1), question_embed.reshape(1, -1))[0][0]
        output_df.at[count, 'question'] = row['question']
        output_df.at[count, 'cosine_similarity'] = cosine_sim
        output_df.at[count, 'dot_product_similarity'] = sim
        output_df.at[count, 'csv_row'] = csv_data
        output_df.at[count, 'index'] = row['index']
        count+=1
        
        # writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # Get the CSV data as a string
    

12it [00:01, 12.05it/s]

2000it [03:05, 10.77it/s]


In [49]:
output_df.to_csv('../embedding_similarity/sentence_transformers/bert_base_nli_mean_tokens.csv', index=False)

: 

In [36]:
model_id = 'RUCKBReasoning/TableLLM-7b'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# with open(filename, 'w') as file:
count = 0
for i, row in tqdm(data[:2000].iterrows()):
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    trows = table.find_all('tr')
    header = trows[0]
    question_embed = tokenizer.encode(row['question'])

    for trow in trows[1:]:
        writer = csv.writer(csv_buffer)
        writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
        csv_data = csv_buffer.getvalue()
        csv_data_embed = tokenizer.encode(csv_data)
        csv_buffer = StringIO()
        sim = np.dot(csv_data_embed, question_embed)
        cosine_sim = cosine_similarity(csv_data_embed.reshape(1, -1), question_embed.reshape(1, -1))[0][0]
        output_df.at[count, 'question'] = row['question']
        output_df.at[count, 'cosine_similarity'] = cosine_sim
        output_df.at[count, 'dot_product_similarity'] = sim
        output_df.at[count, 'csv_row'] = csv_data.replace('\n', '').replace('  ', '')
        output_df.at[count, 'index'] = row['index']
        count+=1
        # writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # Get the CSV data as a string
    

11719

: 

In [38]:
a = [2, 3, 2, 56, 78, 99]
b = [3, 44, 2, 2, 5, 99, 22, 33, 44, 66, 88, 99]

# Count occurrences of elements in list a
a_counts = {}
for item in a:
    a_counts[item] = a_counts.get(item, 0) + 1

# Count occurrences of elements in list b
b_counts = {}
for item in b:
    b_counts[item] = b_counts.get(item, 0) + 1

# Find common elements and count their occurrences
common_values_count = 0
for item, count in a_counts.items():
    if item in b_counts:
        common_values_count += min(count, b_counts[item])

print("Count of the same values in both lists:", common_values_count)

Count of the same values in both lists: 4
